<a href="https://colab.research.google.com/github/c-c-c-c/dm_integration/blob/master/myMecab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# テキストの分類
テキスト分類は、以下の順に行います。

1. 前処理
2. 数値表現化 (CountVectorizer, TfIdfVectorizer)
3. 分類器を学習

本ノートブックでは、Wikipedia のエントリ分類を通じて、テキスト分類器を作成します。

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [1]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-430
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  aptitude-common libcgi-fast-perl libcgi-pm-perl libclass-accessor-perl
  libcwidget3v5 libencode-locale-perl libfcgi-perl libhtml-parser-perl
  libhtml-tagset-perl libhttp-date-perl libhttp-message-perl libio-html-perl
  libio-string-perl liblwp-mediatypes-perl libparse-debianchangelog-perl
  libsigc++-2.0-0v5 libsub-name-perl libtimedate-perl liburi-perl libxapian30
Suggested packages:
  aptitude-doc-en | aptitude-doc apt-xapian-index debtags tasksel
  libcwidget-dev libdata-dump-perl libhtml-template-perl libxml-simple-perl
  libwww-perl xapian-tools
The following NEW packages will be installed:
  aptitude aptitude-common libcgi-fast-perl libcgi-pm-perl
  libclass-accessor-perl libcwidget3v5 libencode-l

In [2]:
!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git
!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a

Cloning into 'mecab-ipadic-neologd'...
remote: Enumerating objects: 75, done.
remote: Counting objects: 100% (75/75), done.
remote: Compressing objects: 100% (74/74), done.
remote: Total 75 (delta 5), reused 54 (delta 0), pack-reused 0
Unpacking objects: 100% (75/75), done.
[install-mecab-ipadic-NEologd] : Start..
[install-mecab-ipadic-NEologd] : Check the existance of libraries
[install-mecab-ipadic-NEologd] :     find => ok
[install-mecab-ipadic-NEologd] :     sort => ok
[install-mecab-ipadic-NEologd] :     head => ok
[install-mecab-ipadic-NEologd] :     cut => ok
[install-mecab-ipadic-NEologd] :     egrep => ok
[install-mecab-ipadic-NEologd] :     mecab => ok
[install-mecab-ipadic-NEologd] :     mecab-config => ok
[install-mecab-ipadic-NEologd] :     make => ok
[install-mecab-ipadic-NEologd] :     curl => ok
[install-mecab-ipadic-NEologd] :     sed => ok
[install-mecab-ipadic-NEologd] :     cat => ok
[install-mecab-ipadic-NEologd] :     diff => ok
[install-mecab-ipadic-NEologd] :   

In [0]:
import joblib
import MeCab
import numpy as np
import pandas as pd
import re
import json
pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 1000)


from collections import Counter
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.svm import LinearSVC

## データの読み込み

In [0]:
# EPGデータ(手作業での修正中))
df = pd.read_excel("./drive/My Drive/0_インテグ作業/data/EPG_checking0212.xlsx")

In [0]:
f= open("./drive/My Drive/0_インテグ作業/data/drama_category0214.json", 'r')

#ココ重要！！
drama_category_dic = json.load(f) #JSON形式で読み込む



In [0]:
f= open("./drive/My Drive/0_インテグ作業/data/drama_win_lose.json", 'r')

#ココ重要！！
drama_win_lose_dic = json.load(f) #JSON形式で読み込む



In [9]:
df.columns

Index(['Unnamed: 0', 'drama_key', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4',
       'drama_title', 'sharp_title', 'sharp_cnt', 'sharp_cnt_epg_regex',
       'start_time', 'sharp_epg', 'sharp_epg_hand_corrected', 'Unnamed: 12',
       'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24'],
      dtype='object')

In [0]:
df["sharp_epg_tknz"] = np.nan

In [0]:
# ゴミ除去

def removeTrash (text):
    import re

    result_text = text
    result_text = re.sub(r"https?://[\w/:%#\$&\?\(\)~\.=\+\-]+", "", result_text)
    result_text = re.sub(r"番組詳細|制作・著作|制作著作", "", result_text)
    result_text = re.sub(r"[!\(\)=『』～/]", "", result_text)
    result_text = re.sub(r"フジテレビ|日本テレビ|TBS|テレビ朝日|TBS|関西テレビ", "", result_text)
    
    result_text = re.sub(r"【公式.*?】", "", result_text)
    result_text = re.sub(r"\u3000", "", result_text)

    return result_text


In [24]:
mecab = MeCab.Tagger()
mecab.parse("")
for i, text in enumerate( df['sharp_epg_hand_corrected']):
    text_tokenized = []
    # print(i)

    # URL、記号などのゴミを取り除く
    if type(text) is not str: 
        if np.isnan(text) :
            continue 
    text = removeTrash(text)
    node = mecab.parseToNode(text)
    while node:
        node = node.next
        if node is None:
            continue

        if not node.feature.startswith("BOS/EOS") and not node.feature.startswith("助詞") and\
            not node.feature.startswith("記号") and\
            node.feature.find("人名") == -1 and\
            not node.feature.startswith("助動詞"):
            text_tokenized.append(node.surface)

    df["sharp_epg_tknz"].iloc[i] = text_tokenized

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
# 手作業での前処理が完了してないので、完了したものだけにする
bool_list = [] 

for i in range(len(df["sharp_epg_hand_corrected"])):
    bool_list.append( type( df["sharp_epg_tknz"].iloc[i] ) != float )

In [0]:
df_notnull =  df[bool_list]

In [27]:
df_notnull["phys_cnt"] = np.nan
df_notnull["category"] = np.nan
df_notnull =  df_notnull.rename(columns={ 'Unnamed: 0' : 'sort_id' } )

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [28]:
drama_category_dic.keys()

dict_keys(['love', 'plice'])

In [29]:
# 物理カウントとカテゴリーを加える
love_cnt = 0
police_cnt = 0
for tmp_key  in df_notnull["drama_key"].unique():
    cnt = 0

    if tmp_key in drama_category_dic["love"]:
        print("love☆")
        love_cnt += 1

        qry_l = " drama_key == @tmp_key"
        target_idx = df_notnull.query(qry_l).index
        df_notnull[ "category" ].loc [target_idx] = "love"


    if tmp_key in drama_category_dic["plice"]:
        police_cnt += 1
        qry_p = " drama_key == @tmp_key"
        target_idx = df_notnull.query(qry_p).index
        df_notnull[ "category" ].loc [target_idx] = "police"
        # print(police_cnt)

    for sort_id in  df_notnull[df_notnull["drama_key"] == tmp_key]['sort_id'].values:
        cnt  += 1

        qry = " sort_id == @sort_id"
        target_idx = df_notnull.query(qry).index
        df_notnull["phys_cnt"].loc[target_idx] = cnt

    print(tmp_key)


# df = pd.DataFrame( columns=['A','B'] )
# 行を作る
# tmp_se = pd.Series( [ i, i*i ], index=list_df.columns )
# # dfに行をレコードとして追加
# df = df.append( tmp_se, ignore_index=True )


/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


1910_CX_月21
1910_CX_火22
love☆
1910_CX_木22
1907_CX_木22
1904_CX_月21
love☆
1904_CX_火21
1904_CX_木22
1901_CX_月21
1901_CX_火21
1901_CX_木22
1810_CX_月21
1810_CX_火21
1810_CX_木22
1807_CX_月21
1807_CX_火21
1807_CX_木22
1804_CX_月21
1804_CX_火21
1804_CX_木22
love☆
1801_CX_月21
1801_CX_木22
1710_CX_月21
1710_CX_火21
1710_CX_木22
1707_CX_火21
1707_CX_木22
1707_CX_日21
1704_CX_月21
1704_CX_火21
1704_CX_木22
1704_CX_日21
1701_CX_月21
1701_CX_火21
1701_CX_木22
1701_CX_日21
1610_CX_月21
1610_CX_火21
1610_CX_木22
1610_CX_日21
1607_CX_月21
1607_CX_火22
1607_CX_日21
1604_CX_月21
1604_CX_火22
1604_CX_木22
1604_CX_土23
love☆
1601_CX_月21
1601_CX_火22
1601_CX_水22
1601_CX_木22
love☆
1510_CX_月21
1510_CX_火22
1510_CX_水22
love☆
1510_CX_木22
love☆
1507_CX_月21
1507_CX_火22
1507_CX_水22
1507_CX_木22
1504_CX_月21
love☆
1504_CX_水22
1504_CX_木22
love☆
1501_CX_月21
1501_CX_火21
1501_CX_火22
1501_CX_水22
1501_CX_木22
1410_CX_月21
1410_CX_火21
1410_CX_火22
1410_CX_水22
1410_CX_木22
1407_CX_月21
1407_CX_火21
1407_CX_火22
1407_CX_水22
love☆
1407_CX_木22
1407_CX_土23
1404_CX_月21
1404

In [30]:
df_notnull[df_notnull["category"] == "love"].drama_key.unique()

array(['1910_CX_木22', '1904_CX_火21', '1801_CX_月21', '1601_CX_月21',
       '1510_CX_月21', '1510_CX_木22', '1507_CX_月21', '1504_CX_水22',
       '1501_CX_月21', '1407_CX_木22', '1404_CX_木22', '1401_CX_月21',
       '1210_CX_木22', '1110_CX_月21', '1110_CX_木22', '1107_CX_月21',
       '1104_CX_月21', '1101_CX_月21', '1010_CX_月21', '1007_CX_月21',
       '1004_CX_月21', '1004_CX_木22', '0907_CX_月21', '0907_CX_火22',
       '0804_CX_火21', '0804_CX_火22', '0801_CX_月21', '0707_CX_月21',
       '1910_EX_土23', '1904_EX_土23', '1901_EX_土23', '1810_EX_土23',
       '1804_EX_土23', '1501_EX_金23', '1004_EX_木21', '1907_NTV_水22',
       '1907_NTV_土22', '1810_NTV_水22', '1807_NTV_水22', '1807_NTV_土22',
       '1804_NTV_木23', '1801_NTV_日22', '1804_TBS_火22', '1801_TBS_火22'],
      dtype=object)

In [0]:
# tmp_key = "1807_TBS_日21"
# qry = "drama_key == @tmp_key & phys_cnt < 4 "
# str(df_notnull.query(qry)["sharp_epg_hand_corrected"].values)

In [0]:
# df = pd.DataFrame( columns=['A','B'] )
# 行を作る
# tmp_se = pd.Series( [ i, i*i ], index=list_df.columns )
# # dfに行をレコードとして追加
# df = df.append( tmp_se, ignore_index=True )


df_stage = pd.DataFrame( columns=['drama_key','drama_title','stages','epg_joined','drama_category','win_lose'] )

# df_notnull["drama_key"].

for tmp_key in df_notnull[df_notnull["category"] == "love"].drama_key.unique():

    win_or_lose = ""
    if tmp_key in drama_win_lose_dic["win"]:
        win_or_lose = "win"
    elif tmp_key in drama_win_lose_dic["draw"]:
        win_or_lose = "draw"
    elif tmp_key in drama_win_lose_dic["lose"]:
        win_or_lose = "lose"
    else:
        print("????")

        target_idx = df_stage.query(qry_wl).index
        df_notnull[ "category" ].loc [target_idx] = "police"
        # print(police_cnt)


    for tmp_stage in ["early","middle" ,"late"]:

        qry = ""
        if tmp_stage == "early":
            qry = "drama_key == @tmp_key & phys_cnt < 4 "

        elif tmp_stage == "middle":
            qry = "drama_key == @tmp_key & 4< phys_cnt < 7 "

        else:
            qry = "drama_key == @tmp_key & phys_cnt >= 7 "

        target_i =  df_notnull.query(qry)["sharp_epg_hand_corrected"]
        epg_joined = str(df_notnull.query(qry)["sharp_epg_hand_corrected"].values)

        tmp_se = pd.Series( [ tmp_key, "..title" ,tmp_stage, epg_joined  ,"love", win_or_lose ]  , index=df_stage.columns  )
        df_stage = df_stage.append( tmp_se, ignore_index=True )

# early, middle, lateで作る

# for i in range():
    




In [33]:
df_stage


,drama_key,drama_title,stages,epg_joined,drama_category,win_lose
0,1910_CX_木22,..title,early,['現在失業中の27歳独身・難波ユリカ(新木優子)は、見た目にも気を使い、コミュニケーション...,love,lose
1,1910_CX_木22,..title,middle,['マコチ(高良健吾)は、ユリカ(新木優子)のもとへ謝りに行ったものの、追い返されてしまう。...,love,lose
2,1910_CX_木22,..title,late,['ユリカ(新木優子)は、マコチ(高良健吾)から「好きだ」と告白され、もう一度彼と付き合うこ...,love,lose
3,1904_CX_火21,..title,early,['12年ぶりの再会は偶然か、それとも運命かーー。恋をあきらめた鮎川樹(松坂桃李)と、初恋相...,love,win
4,1904_CX_火21,..title,middle,['つぐみ(山本美月)が駅のホームから転落した。車いすから懸命に手を伸ばすが届かず、線路に落...,love,win
5,1904_CX_火21,..title,late,['松本に戻って3カ月、つぐみ(山本美月)はインテリアコーディネーターとして働きながら、実家...,love,win
6,1801_CX_月21,..title,early,"['クラゲを愛しすぎてしまった筋金入りの“クラゲオタク女子""が、とある兄弟と三角関係になり、...",love,lose
7,1801_CX_月21,..title,middle,['稲荷翔子(泉里香)が『天水館』の持ち主である千絵子(富山えり子)の母に直接、土地建物の売...,love,lose
8,1801_CX_月21,..title,late,"['修(工藤阿須加)から告白された月海(芳根京子)は大混乱。“尼～ず""に相談出来ない月海は、...",love,lose
9,1601_CX_月21,..title,early,['幼いころに女手ひとつで育ててくれた母を亡くした杉原音(有村架純)は、育ての親である林田雅...,love,draw


In [0]:
# 空の列を足す
df_stage["epg_tknz"] = np.nan

In [35]:
mecab = MeCab.Tagger()
mecab.parse("")
for i, text in enumerate( df_stage['epg_joined']):
    text_tokenized = []
    # print(i)

    # URL、記号などのゴミを取り除く
    if type(text) is not str: 
        if np.isnan(text) :
            continue 
    text = removeTrash(text)
    node = mecab.parseToNode(text)
    while node:
        node = node.next
        if node is None:
            continue

        if not node.feature.startswith("BOS/EOS") and not node.feature.startswith("助詞") and\
            not node.feature.startswith("記号") and\
            node.feature.find("人名") == -1 and\
            not node.feature.startswith("助動詞"):
            text_tokenized.append(node.surface)

    df_stage["epg_tknz"].iloc[i] = text_tokenized

/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


In [0]:
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")

In [0]:

X = vectorizer.fit_transform(
    [str(i) for i in df_stage["epg_tknz"].values]
)

In [38]:
X

<132x6829 sparse matrix of type '<class 'numpy.float64'>'
	with 25239 stored elements in Compressed Sparse Row format>

In [39]:
pd.DataFrame(X.toarray(), columns=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ])

,02,1,10,100,110,12,14,15,18,19,2,20,2003,2009,2011,2013,2014,2018,2019,22,24,27,29,3,30,3000,30001,30002,30002008,30002015,30006,30007,31,32,33,35,36,4,40,44,5,50,6,60,6000,7,70,8,ak,ap,arata,asamoto,b,bar,bl,bonito,c,ca,caldo,cd,ceo,ct,cube,ecu,ela,f,fifa,gps,id,jazz,jc,jmt,keiji,mkm,no,ok,pba,pc,ring,smbc,sns,tap,tシャツ,u,vs,vtr,w,web,you,zukan,あ,あいさつ,あいさつ回り,あう,あえて,あおり,あおる,あがめる,あきらめ,あきらめる,あきれ,あきれる,あきれ返る,あくる,あくる日,あぐね,あげ,あげよ,あげる,あしらう,あっ,あっけ,あっけなく,あっさり,あて,あてつけ,あてつける,あと,あなた,あの,あふれる,あまり,あまりに,あやしい,あやしむ,あら,あらた,あらためて,あらわ,あり,ありがとう,ありのまま,ある,あれ,あれこれ,あろ,あわせ,あんた,あんなに,い,いい,いい加減,いう,いえ,いか,いかに,いかにも,いき,いきなり,いきりたっ,いく,いくら,いけ,いける,いさめ,いじめ,いたずら,いただき,いただける,いたる,いたわる,いた事,いちゃつく,いち早く,いっ,いっこうに,いったい,いったん,いっぱい,いつ,いつか,いつしか,いつも,いと,いとこ,いま,いまだ,いやらしい,いよ,いよいよ,いら立ち,いり,いる,いれる,いろいろ,いわゆる,う,うえ,うかがい,うそ,うた,うち,うっ,うっかり,うっとうしく,うとまし,うながさ,うながす,うなされ,うまい,うまく,うまれ,うれしい,うろうろ,うろつい,うろつく,うんざり,うんと,え,えみ,えり,えれ,お,おい,おいしい,おかげ,おかしい,おかしな,おき,おく,おけ,おこ,おごり,おごる,おしゃべり,おじいちゃん,おじけづい,おじけづく,おせっかい,おそらく,おっさん,おなか,おばさん,おびえる,おぼつかない,おまえ,おもてなし,おり,おんぶ,お互い,お似合い,お供,お前,お化け,お子さま,お手伝い,お披露目,お母さん,お気にいり,お礼,...,陵,陶芸,陽,陽子,階,階下,隙,際,障,障害,隠し,隠し事,隠し子,隠し持つ,隠す,隠せ,隠れ,隣,隣り,隣人,雄,雄大,雅,集い,集う,集まっ,集まり,集まる,集中,集合,雇う,雇っ,雇わ,雇用,雉畑,雑巾,雑木林,雑誌,離す,離せ,離れ,離れる,離れ離れ,離婚,難しい,難しく,難波,難癖,難航,難色,難関,雨,雨宿り,雪,雪が谷大塚,雫,雰囲気,電報,電源,電話,電車,震え,震える,青年,青木,静,静か,静まり返る,非常勤,非難,面,面々,面し,面会,面倒,面倒くさい,面持ち,面接,面白い,面談,面識,靴,鞄,韓国,音,音声,音大,音楽,音楽家,響,響く,頃,順番,順調,預かっ,預かり,預かる,預ける,頑張っ,頑張り,頑張る,頓挫,頬,頭,頭巾,頭文字,頭痛,頻繁,頼っ,頼ま,頼み,頼みこむ,頼む,頼り,頼る,頼ん,顎,顔,顔なじみ,顔色,願い,願う,願っ,顧問,顧客,風,風呂,風景,風見,風邪,飛ば,飛ばさ,飛ばし,飛び入り,飛び出し,飛び出す,飛び込む,飛び込ん,飛び降りよ,飛行機,食,食いしばる,食い下がり,食い下がる,食い入る,食べ,食べる,食事,食卓,食品,食器,食材,飯田,飯豊,飲み,飲む,飲ん,飼い,飼う,飼っ,飼育,飾ら,飾り,館,首,香炉,馬,馬場,馳,馳天,馴れ初め,駅,駅前,駆け,駆けつけ,駆けつける,駆け付ける,駆け出し,駆け寄る,駆け落ち,駆け込み,駆け込む,駆ら,駐車,騒が,騒ぎ,騒ぎ立て,騒動,騙し,驚い,驚か,驚かす,驚がく,驚き,驚く,驚愕,高,高く,高まる,高め,高山,高嶺,高性能,高揚,高校,高校生,高梨,高等,高級,高速,高飛車,高鳴る,髪,鬼,鬼気,魅了,魅力,魔,魔女,魚,鮮明,鯉淵,鳴か,鳴っ,鳴ら,鳴らす,鳴り,鳴り響い,鳴る,鶏,鶴,鶴岡,麦,麻衣,黒,黒幕,黙々と,黙っ,黙ら,黙り込ん,鼻,鼻血,龍,０,１,２,３,４,５,７
0,0.000000,0.028738,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.011933,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.027717,0.000000,0.016440,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.029488,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.029488,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.013396,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023452,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.017391,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.034713,0.014731,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038883,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.038337,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.023452,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0

In [40]:
vectorizer.idf_

array([5.19720195, 2.33500107, 3.81090759, ..., 5.19720195, 4.79173684,
       5.19720195])

In [0]:
# idf値の確認

idf = pd.Series(vectorizer.idf_, index=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ]) \
    .to_frame("idf") \
    .sort_values("idf", ascending=False)

In [42]:


pd.set_option('display.max_rows', 1000)
idf.iloc[1:100]

,idf
懸念,5.197202
戦々恐々,5.197202
我が,5.197202
成田空港,5.197202
成果,5.197202
成就,5.197202
成り上がり,5.197202
懸命,5.197202
戦友,5.197202
懸け,5.197202


In [0]:
#####################
#
#以下、授業の残り
#
####################



## 形態素解析
---

MeCab を利用した形態素解析と、簡単な処理の例を示します。

### 簡単な例

In [0]:
# ※これは、演習用に単語文書行列を DataFrame に変換して見やすくしてみるためのコードで、覚える必要はありません
pd.DataFrame(X.toarray(), columns=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ])

CountVectorizer が生成する単語文書行列では、行 (横) 方向が1つの文書を表し、それぞれの列に単語の出現回数が格納されます (※単語文書行列と言った場合、列方向が文書を表す場合もあるので、他の文献を読む際は注意が必要です)。

どの列がどの単語と対応しているかを知るためには、vectorizer の vocabulary_ を参照します。

In [0]:
vectorizer.vocabulary_

新しい文書に対しては、transform で変換します。

なお、fit\_transform の際に出てこなかった単語 (= vocabulary_ にない単語) については、行列内に登場しません。

In [0]:
# transform も、リスト形式で与える必要があるので注意
X_new = vectorizer.transform([
    "Hello I like this pen"
])
X_new.toarray()

### 日本語データに適用
前処理済みの日本語データに適用してみます。DataFrame にスペース区切りのテキストを準備している場合、DataFrame の列を参照させたものをそのまま与えることができます。

In [0]:
vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
X = vectorizer.fit_transform(df["text_tokenized"])

In [0]:
X

In [0]:
X[0].toarray()

↑この規模の単語文書行列になると、行列のサイズ (文書数x単語数) と比較し、値が入っている場所をほとんど無いことが分かります。

In [0]:
vectorizer.vocabulary_

## TF-IDF変換
---

TF-IDF 変換を行うと、単語の出現回数に、他の文書全体と比較した際の希少性 (レア度) で重みを付けることができます。

TF-IDF 変換済みの単語文書行列は、`TfidfVectorizer` で作成できます。

### 簡単な例

In [0]:
# token_pattern は、デフォルトだと1文字の単語を除外するので、除外しないように設定する
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")

In [0]:
df[""]

In [0]:
# スペース区切りの文書をリストで与える
X = vectorizer.fit_transform([
    "This is a pen",
    "I am not a pen",
    "Hello"
])
X.toarray()

In [0]:
# ※これは、演習用に単語文書行列を DataFrame に変換して見やすくしてみるためのコードで、覚える必要はありません
pd.DataFrame(X.toarray(), columns=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ])

他の文書でも出現している単語については、同じ出現回数でも重みが低くなっていることが分かります。各単語の重みは、`idf_` で見ることができます。確認してみましょう。

In [0]:
vectorizer.idf_

In [0]:
# ※これは、演習用に IDF 値を DataFrame に変換して見やすくしてみるためのコードで、覚える必要はありません
pd.DataFrame(np.atleast_2d(vectorizer.idf_), columns=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ])

デフォルトでは、各文書のノルムが1になるように正規化されています (norm="l2")。確認してみましょう。

In [0]:
np.linalg.norm(X.toarray(), axis=1)

### 日本語データに適用
CountVectorizer の場合と同様に、前処理済みの日本語データに適用してみます。

In [0]:
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
X = vectorizer.fit_transform(df["text_tokenized"])

In [0]:
X[0].toarray()

IDF 値の大きい単語・小さい単語を確認してみましょう。

In [0]:
# ※これは、演習用に IDF 値を DataFrame に変換して見やすくしてみるためのコードで、覚える必要はありません
pd.Series(vectorizer.idf_, index=[ x[0] for x in sorted(vectorizer.vocabulary_.items(), key=lambda x: x[1]) ]) \
    .to_frame("idf") \
    .sort_values("idf", ascending=False)

## テキスト分類
---
定量化したテキスト情報を使うことで、テキストをカテゴリー別に分類してみましょう

In [0]:
# テストデータ
df_test = pd.read_csv("dataset/wikipedia-test.txt", sep="\t")
df_test.head()

### 単語文書行列 (TF-IDF 適用済み) の作成
単語文章行列を作成する前に、まずは文章を形態素解析します。

In [0]:
mecab = MeCab.Tagger("-O wakati")

text_tokenized = []
for text in df_test['text']:
    text_tokenized.append(mecab.parse(text))
    
df_test['text_tokenized'] = text_tokenized

続いて、単語文章行列を学習データテストデータでそれぞれ生成します。<br>
この時、2つの単語文章行列は同じ次元数でなければならないので、`transform`を使用します

In [0]:
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
X = vectorizer.fit_transform(df["text_tokenized"])

In [0]:
X_test = vectorizer.transform(df_test["text_tokenized"])

### 分類器の適用・比較
単語文書行列ができたら、後はこれまでやってきた機械学習と同様です。

いくつかの分類器で、性能を比較してみましょう。

In [0]:
# Accuracy, Precision/Recall/F-score/Support, Confusion Matrix を表示
def show_evaluation_metrics(y_true, y_pred):
    print("Accuracy:")
    print(accuracy_score(y_true, y_pred))
    print()
    
    print("Report:")
    print(classification_report(y_true, y_pred))
    
    print("Confusion matrix:")
    print(confusion_matrix(y_true, y_pred))

### ロジスティック回帰

In [0]:
clf_lr = LogisticRegression(n_jobs=-1)
clf_lr.fit(X, df["category"])

推定結果は predict で得ることができます (他の分類器でも同様)。

In [0]:
clf_lr.predict(X_test)

In [0]:
y_test_pred = clf_lr.predict(X_test)
show_evaluation_metrics(df_test["category"], y_test_pred)

### SVM (線形カーネル)

In [0]:
clf_svc = LinearSVC()
clf_svc.fit(X, df["category"])

In [0]:
y_test_pred = clf_svc.predict(X_test)
show_evaluation_metrics(df_test["category"], y_test_pred)

### Random Forest

In [0]:
clf_rf = RandomForestClassifier(n_estimators=50, n_jobs=-1)
clf_rf.fit(X, df["category"])

In [0]:
y_test_pred = clf_rf.predict(X_test)
show_evaluation_metrics(df_test["category"], y_test_pred)

## 【演習1】
上記いずれかのアルゴリズムのパラメータを変更して、分類精度を高めてみましょう。

In [0]:
# Enter your code here

### モデルの保存
`pickle` でも保存できますが、`joblib` を使い、ファイル名末尾に `.gz`, `.bz2` 等を指定すると、自動的に圧縮してくれます。読み込み時も同様、自動的に解凍して読み込んでくれます。

言語処理ではモデルが大容量になることが多いため、モデルの圧縮は重要です。

In [0]:
joblib.dump(clf_rf, "wikipedia_category_classifier.pkl.gz")

### モデル読み込みテスト

In [0]:
clf_rf_restored = joblib.load("wikipedia_category_classifier.pkl.gz")
clf_rf_restored

読み込んだモデルは、上記ですでに学習済みなので、`predict`で予測が行えます

In [0]:
clf_rf_restored.predict(X_test)